Importing basic libraries

In [2]:
import numpy as np
import pandas as pd

Fetch the processed file (this is the file which we saved after processing the original dataset)

In [3]:
df= pd.read_csv('/content/EmotionAnalysis/main/saved data/processed data.csv')

In [4]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4395649 sha256=e3a8641bc4eecabd81983135ce029798215ae7282850f4b95125394101b91f87
  Stored in directory: /root/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


Generating vector representations for the text using fasttext

In [5]:
import fasttext
un_model = fasttext.train_unsupervised('/content/EmotionAnalysis/main/saved data/text.txt')

adding a column in the dataframe of vectorized representation of the input text

In [6]:
df.text = df.text.astype(str)
inputs = []
for texts in df['text']:
    inputs.append(un_model.get_sentence_vector(texts))

df['Inputs'] = inputs

Since the data is imbalanced we apply SMOTE to oversample the minority classes

In [7]:
from imblearn.over_sampling import SMOTE

X = np.array([input for input in df['Inputs']])
y = np.array(df['Emotions'])

smt = SMOTE(random_state=42)
X, y = smt.fit_resample(X, y)

spliting the data into train and test

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, shuffle=True)

training the data with Random Forest Classifier

In [9]:
from sklearn.ensemble import RandomForestClassifier


RFC = RandomForestClassifier(n_estimators = 100)  
RFC.fit(X_train, y_train)

RandomForestClassifier()

Getting the accuracy score and the f1 score of the model

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

y_pred = RFC.predict(X_test)
accuracy= accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred , average=None)

print(accuracy)
print(f1)

0.9615038323021516
[0.92668717 0.96613134 0.96509626 0.96204567 0.93780975 0.96278473
 0.96714514 0.96535358 0.98401305 0.98222366 0.94447797 0.98500797
 0.9955836  0.97871665 0.99499758 0.96067775 0.99790559 0.97916998
 0.97232213 0.99699889 0.38635798 0.97341188 0.99773756 0.97933884
 0.99759423 0.99333442 0.98732927 0.97776376]


Saving the model

In [11]:
import pickle
with open('model_pkl', 'wb') as files:
    pickle.dump(RFC, files)